In [49]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv 
import requests
import pandas as pd
import re
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import urllib.request
import random
import numpy as np
import os
import warnings 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import NoSuchFrameException
from selenium.common.exceptions import NoAlertPresentException

warnings.filterwarnings(action='ignore') #경고 무시

<식기세척기_커뮤니티 원문_세대별_21.1_21.12.csv>  
총 row: 17,309  
collected_community 기반 총 row: 17,303  


* 제거 URL
https://www.instiz.net/name_enter/77131392?&category=2

In [50]:
url_file = pd.read_csv('./data_file/식기세척기_커뮤니티 원문_세대별_21.1_21.12.csv', encoding="cp949")
community_list_file = pd.read_csv('./data_file/community_list (2).csv', encoding='cp949')
collected_community = community_list_file['COMMUNITY'].to_list()
collected_community_df = url_file[url_file['URL'].apply(lambda x: x[x.find('/')+2:x.find('/', 8)]).isin(collected_community)]

In [51]:
len(url_file)

17308

In [52]:
url_file.head()

,No.,채널,날짜,작성자,제목,URL,세대
0,1,인스티즈3,20210126,NaN,우리가 개인 텀블러 가져와서 물먹거든? \r\n근데 매일 마감 때 식기 세척기 돌리...,http://www.instiz.net/name/41093761?category=1,10대
1,2,인스티즈3,20210126,NaN,하지만 지금 내 자취방엔 행거 하나 둘 자리도 없지ㅠ,http://www.instiz.net/name/41091139?category=1,10대
2,3,인스티즈3,20210125,NaN,이성 사랑방 에 게시된 글입니다 \r\n둘 다 서울사람인데 내가 회사사정으로 대전...,http://www.instiz.net/name/41071039?category=1,10대
3,4,인스티즈3,20210121,NaN,아니 알바하는데 다같이 위시하는가 식기세척기 안쓰고 똑바로 워시하기로 했으면서 식시...,http://www.instiz.net/name/40983623?category=1,10대
4,5,인스티즈3,20210117,NaN,그릇 씻기는거보면 속시원,http://www.instiz.net/name/40906917?category=1,10대


In [53]:
# 제거된 커뮤니티
url_file[url_file['URL'].apply(lambda x: x[x.find('/')+2:x.find('/', 8)]).isin(collected_community)]

,No.,채널,날짜,작성자,제목,URL,세대
0,1,인스티즈3,20210126,NaN,우리가 개인 텀블러 가져와서 물먹거든? \r\n근데 매일 마감 때 식기 세척기 돌리...,http://www.instiz.net/name/41093761?category=1,10대
1,2,인스티즈3,20210126,NaN,하지만 지금 내 자취방엔 행거 하나 둘 자리도 없지ㅠ,http://www.instiz.net/name/41091139?category=1,10대
2,3,인스티즈3,20210125,NaN,이성 사랑방 에 게시된 글입니다 \r\n둘 다 서울사람인데 내가 회사사정으로 대전...,http://www.instiz.net/name/41071039?category=1,10대
3,4,인스티즈3,20210121,NaN,아니 알바하는데 다같이 위시하는가 식기세척기 안쓰고 똑바로 워시하기로 했으면서 식시...,http://www.instiz.net/name/40983623?category=1,10대
4,5,인스티즈3,20210117,NaN,그릇 씻기는거보면 속시원,http://www.instiz.net/name/40906917?category=1,10대
...,...,...,...,...,...,...,...
17303,136,은진마을,20211202,가을***,씽크대 하부장을 설치하려고 합니다. \r\n서랍장 또는 칸막이 하고 문짝 설치하신 ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...,연령불가
17304,137,울산부동산재테크,20211202,아옹***,오븐기 자리에 식기세척기 설치하고 \r\n렌지장 밥솥위에 오븐기를 매립 하려구요 \...,https://cafe.naver.com/ArticleRead.nhn?clubid=...,연령불가
17305,138,오븐엔조이,20211201,o할****,수원 신동에 위치하고 있고 \r\n신동에 드문 17평대 매물입니다. \r\n매장과 ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...,연령불가
17306,139,아름다운 귤현동 사람들,20211201,사랑***,맘님들께서 애정하는 아이템 무엇이 있는 가요? \r\n? \r\n저같은 경우에는첫 ...,https://cafe.naver.com/ArticleRead.nhn?clubid=...,연령불가


In [54]:
naver_url_df = url_file[url_file['URL'].apply(lambda x: x[x.find('/')+2:x.find('/', 8)]).isin(['cafe.naver.com'])]
rest_url_df = url_file[~url_file['URL'].apply(lambda x: x[x.find('/')+2:x.find('/', 8)]).isin(['cafe.naver.com'])]

In [55]:
url_file['URL'].apply(lambda x: x[:x.find('/', 8)]).unique()

array(['http://www.instiz.net', 'http://www.hungryapp.co.kr',
       'https://pann.nate.com', 'https://cafe.naver.com',
       'https://hiphople.com', 'https://forum.blackdesertm.com',
       'http://www.slrclub.com', 'https://ohou.se',
       'http://gall.dcinside.com', 'https://www.dmitory.com',
       'https://www.fmkorea.com', 'https://theqoo.net',
       'http://cafe.daum.net', 'http://www.gasengi.com',
       'http://www.inven.co.kr', 'https://gall.dcinside.com',
       'https://bbs.ruliweb.com', 'http://www.etoland.co.kr',
       'http://web.humoruniv.com', 'https://www.inven.co.kr',
       'http://nbamania.com', 'http://www.oeker.net',
       'https://www.dogdrip.net', 'http://www.ppomppu.co.kr',
       'http://mlbpark.donga.com', 'https://m.clien.net',
       'http://www.bobaedream.co.kr', 'https://www.mimint.co.kr',
       'http://www.todaysppc.com', 'http://www.todayhumor.co.kr',
       'http://eunpyeong.momscafe.net', 'https://www.ppomppu.co.kr',
       'http://www.paxnet.c

------------------------------------------------
#### <b> 커뮤니티 마다 원문 가져오기<b>

In [56]:
def soccerline_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.titBox')[0].text.strip()
        review = soup.select('.txtBox')[0].text.strip()
        iscomment = soup.select('.replyTxt')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').replace('\xa0',' ').strip())
        else:
            box = []
            pass
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# soccerline_contents('https://soccerline.kr/board/16751915?searchWindow=&searchType=0&searchText=&categoryDepth01=0&page=11')

In [57]:
def dealbada_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('#bo_v_info > div > span')[0].text.strip()
        review = soup.select('#bo_v_con')[0].text.strip()
        iscomment = soup.select('#bo_vc > table > tbody > tr > td > div:nth-child(2) > p')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').replace('\xa0',' ').strip())
        else:
            box = []
            pass
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# dealbada_contents('http://www.dealbada.com/bbs/board.php?bo_table=comm_free&wr_id=2553878')

In [58]:
def thinkpool_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('#printSubject > div.view_title > strong')[0].text.strip()
        review = soup.select('.viewContent > div')[0].text.strip()
        box = []
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# thinkpool_contents('https://www.thinkpool.com/compDiscuss/cont/961927?sn=961927&pageNo=2&iMax=00009619859999')

In [59]:
def goso_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('body > table:nth-child(6) > tbody > tr > td > table > tbody > tr > td > table:nth-child(5) > tbody > tr > td > div:nth-child(2) > table > tbody > tr > td:nth-child(1) > div')[0].text.strip()
        review = soup.select('#writeContents')[0].text.strip()
        box = []
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# goso_contents('http://www.goso.co.kr/bbs/board.php?bo_table=testDB&wr_id=1055944&sca=&sfl=wr_subject%7C%7Cwr_content&stx=%EC%B9%B4%EB%93%9C&sop=and')

In [60]:
def koreapork_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.vw_tit')[0].text.strip()
        review = soup.select('#zoom_box > table > tbody > tr > td')[0].text.strip()
        box = []
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
# koreapork_contents('https://www.koreapork.or.kr/board/view1?Ncode=m8&number=69211')

In [61]:
def dprime_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('#writeSubject')[0].text.strip()
        title = title.split(']')[1].strip()
        review = soup.select('#resContents')[0].text.strip()
        iscomment = soup.select('.comment_content')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').replace('\xa0',' ').strip())
        else:
            box = []
            pass
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# dprime_contents('https://dprime.kr/g2/bbs/board.php?bo_table=comm&wr_id=22440061')

In [62]:
def ddanzi_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('div.top_title > h1 > a.link')[0].text.strip()
        review = soup.select('.read_content')[0].text.strip()
        iscomment = soup.select('.fdComment')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').replace('\xa0',' ').strip())
        else:
            box = []
            pass
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# ddanzi_contents('https://www.ddanzi.com/index.php?mid=free&document_srl=665689062')

In [63]:
def cook82_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('#column2 > h4 > span:nth-child(2)')[0].text.strip()
        review = soup.select('#articleBody')[0].text.strip()
        iscomment = soup.select('.rp > p')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').replace('\xa0',' ').strip())
        else:
            box = []
            pass
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# cook82_contents('https://www.82cook.com/entiz/read.php?bn=15&num=3520837&page=1')

In [64]:
def eunpyeong_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.board-view-title')[0].text.strip()
        review = soup.select('.board-view-body')[0].text.strip()
        iscomment = soup.select('.comment')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').replace('\xa0',' ').strip())
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# eunpyeong_contents('http://eunpyeong.momscafe.net/board/view/talk/all/249096?page=2')

In [65]:
def todayhumor_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.viewSubjectDiv')[0].text.strip()
        review = soup.select('.viewContent')[0].text.strip()
        iscomment = soup.select('.memoContent')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').replace('\xa0',' ').strip())
        else:
            box = []
            pass
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# todayhumor_contents('http://www.todayhumor.co.kr/board/view.php?table=baby&no=25034&s_no=25034&page=2')

In [66]:
def todaysppc_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return
    try:
        title = soup.select('div.main > table > tbody > tr:nth-child(2) > td:nth-child(2) > table > tbody > tr > td:nth-child(1) > h1 > font > b')[0].text.strip()
        review = soup.select('#content_view')[0].text.strip()
        box = []
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# todaysppc_contents('http://www.todaysppc.com/mbbs/view.php?id=free&no=456508')

In [67]:
def mimint_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.board_view_info > h2')[0].text.strip()
        review = soup.select('#board_content')[0].text.strip()
        iscomment = soup.select('#BoardRripleList_Space > table> tbody > tr > td > table > tbody > tr > td')
        
        if len(iscomment) != 0:
            box = []
            cnt=0
            for i in iscomment:
                if cnt % 8 == 3:
                    box.append(i.text.replace('\n','').replace('\t','').replace('\xa0','').strip())
                cnt += 1
        else:
            box = []
            pass
                
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return

In [68]:
def clien_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.post_subject > span')[0].text.strip()
        review = soup.select('.post_article.inline_link')[0].text.strip()
        iscomment = soup.select('.comment_view')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                i_text = i.text.replace('\n','').strip()
                golbangi = i_text.find("@")
                re_id = i_text[golbangi : i_text.find("님", golbangi)+1]
                box.append(i_text.replace(re_id, "").strip())
        else:
            box = []
            pass
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# clien_contents('https://m.clien.net/service/board/park/15842272')

In [69]:
def etoland_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.title')[0].text.strip()
        review = soup.select('#view_content')[0].text.strip()
        iscomment = soup.select('.mw_basic_comment_content')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').strip())
        else:
            box = []
            pass
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)        
        
    except:
        return
    
# etoland_contents('https://www.etoland.co.kr/bbs/board.php?bo_table=etohumor05&wr_id=161990')

In [70]:
def ruliweb_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.subject_inner_text')[0].text.strip()
        review = soup.select('.view_content.autolink')[0].text.strip()
        iscomment = soup.select('.comment > div > .text')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').strip())
        else:
            box = []
            pass
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# ruliweb_contents('https://bbs.ruliweb.com/hobby/board/300117/read/30635218')

In [71]:
def dmitory_contents(url, time, age, commu):  # 비회원 댓글 열람 불가
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.top_area.ngeb > h1 > a')[0].text.strip()
        review = soup.select('.read_body > div')[0].text.strip()
        box=[]
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# dmitory_contents('https://www.dmitory.com/index.php?mid=roommate&document_srl=169136548')

In [72]:
def ohou_contents(url, time, agem, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        menu = url[url.find('/',8)+1 : url.find('/',16)]
        if menu == 'contents':
            title = ""
            review_tmp = ""
            review = soup.select('.css-mjr9jp')
            for r in review:
                review_tmp = review_tmp + " " + r.text
            box = []
            
        else: # 질문과 답변
            title = soup.select(".css-15u7kw5")[0].text
            review = soup.select('.css-yquyes.e76tv1e2')[0].text
            iscomment = soup.select('.css-eayw7z > p')
            if len(iscomment) != 0:
                box = []
                for i in iscomment:
                    i_text = i.text.replace('\n','').strip()
                    golbangi = i_text.find("@")
                    re_id = i_text[golbangi : i_text.find("\xa0", golbangi)+1]
                    box.append(i_text.replace(re_id, ""))
                        
            else:
                box = []
                pass
            
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# ohou_contents('https://ohou.se/questions/66666?affect_id=0&affect_type=QuestionIndex&query=')

In [73]:
def blackdesertm_contents(url, time ,age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.boardHead > strong')[0].text.strip()
        review = soup.select('#wrapper > div > div.container > div.boardWrapper > div.boardDetail > div.boardContent > div')[0].text.strip()
        iscomment = soup.select('.replyText')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').strip())
        else:
            box = []
            pass
        
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# blackdesertm_contents('https://forum.blackdesertm.com/Board/Detail?boardNo=12&contentNo=487485')

In [74]:
def hiphople_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.ah-title')[0].text.strip()
        review = soup.select('#flagArticle > div')[0].text.strip()
        iscomment = soup.select('.comment-body > div > p')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').strip())
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# hiphople_contents('https://hiphople.com/workroom/21422675')

In [102]:
def inven_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('.articleTitle')[0].text.strip()
        review = soup.select('.articleContent')[0].text.strip()
        iscomment = soup.select('.content.cmtContentOne')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').replace('\xa0',' ').strip())
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# inven_contents('http://www.inven.co.kr/board/webzine/2097/1610309?iskin=webzine','1','1','1')

In [76]:
def gasengi_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html')

    except:
        return

    try:
        title = soup.select('#view_conta > table > tbody > tr > td > div > table > tbody > tr')[0].text.strip()
        review = soup.select('#writeContents_sier')[0].text.strip()
        iscomment = soup.select('#commentContents > table > tbody > tr > td > table > tbody > tr > td > div')
        if len(iscomment) != 0:
            box = []
            cnt = 0
            for i in iscomment:
                if cnt % 2 == 1:
                    box.append(i.text.replace('\n','').strip())
                cnt += 1
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# gasengi_contents('http://www.gasengi.com/main/board.php?bo_table=commu&wr_id=3147915')

In [77]:
def theqoo_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html.parser')

    except:
        return

    try:
        title = soup.select('.title')[0].text.strip()
        review = soup.select('.rd_body')[0].text.strip()
        iscomment = soup.select('.fdb_itm.clear')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                try:
                    minitext_element = i.select('.meta.virtual_comment')[0]
                    minitext_element.extract()
                    box.append(i.text.replace('\n','').strip())
                except:
                    box.append(i.text.replace('\n','').strip())
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# theqoo_contents('https://theqoo.net/index.php?mid=review&filter_mode=normal&document_srl=1807810419')
#https://theqoo.net/index.php?mid=review&filter_mode=normal&document_srl=1814998910


In [78]:
def bobae_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    #프레임 전환
    try: 
        html_source = driver.page_source
        soup = bs(html_source, 'html.parser')

    except:
        return

    try:
        title = soup.select('#print_area > div.writerProfile > dl > dt > strong')[0].text.strip()
        review = soup.select('.bodyCont')[0].text.strip()

        iscomment = soup.select('#cmt_reply > li > dl > dd')
        if len(iscomment) != 0:
            box = []
            for i in iscomment:
                box.append(i.text.replace('\n','').replace('\t','').strip())
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# bobae_contents('http://www.bobaedream.co.kr/view?code=freeb&No=2269299&bm=1')
# bobae_contents('https://www.bobaedream.co.kr/view?code=strange&No=4360128&bm=1')

In [79]:
def slr_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    try:
        html_source = driver.page_source
        soup = bs(html_source, 'html.parser')
    except:
        return
    
    try:
        title = soup.select('.sbj')[0].text.strip()
        review = soup.select('#userct')[0].text.strip()

        iscomment = soup.select('.cmt-contents')
        if len(iscomment) != 0:
                box = []
                for i in iscomment:
                    try:
                        minitext_element = i.select('.re_prtnm')[0]
                        minitext_element.extract()
                        box.append(i.text.replace('\n','').strip())
                    except:
                        box.append(i.text.replace('\n','').strip())
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# slr_contents('http://www.slrclub.com/bbs/vx2.php?id=free&no=38970797')

In [80]:
def fmkorea_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    try:
        html_source = driver.page_source
        soup = bs(html_source, 'html.parser')
    except:
        return
    
    try:
        title = soup.select('.np_18px_span')[0].text.strip()
        review = soup.select('.xe_content')[0].text.strip()

        iscomment = soup.select('.fdb_itm > div > div')
        if len(iscomment) != 0:
                box = []
                for i in iscomment:
                    try:
                        minitext_element = i.select('.findParent')[0]
                        minitext_element.extract()
                        box.append(i.text.strip())
                    except:
                        box.append(i.text.strip())
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
        
    except:
        return
    
# fmkorea_contents('https://www.fmkorea.com/4026236722')
# fmkorea_contents('https://www.fmkorea.com/4024409838')

In [81]:
def humoruniv_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    try:
        html_source = driver.page_source
        soup = bs(html_source, 'html.parser')
    except:
        return
    
    try:
        title = soup.select('#ai_cm_title')[0].text.strip()
        review = soup.select('#ai_cm_content')[0].text.strip()

        iscomment = soup.select('.cmt_list')
        if len(iscomment) != 0:
                box = []
                for i in iscomment:
                    box.append(i.text.replace('\n','').replace('\t',''))
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
    
    except:
        return
# humoruniv_contents('http://web.humoruniv.com/board/humor/read.html?table=free&number=4177774')

In [82]:
def mania_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    try:
        html_source = driver.page_source
        soup = bs(html_source, 'html.parser')
    except:
        return
    
    try:
        title = soup.select('#writeSubject')[0].text.strip()
        review = soup.select('#resContents')[0].text.strip()

        iscomment = soup.select('.comment_content.clear')
        if len(iscomment) != 0:
                box = []
                for i in iscomment:
                    box.append(i.text.replace('\n','').replace('\xa0',''))
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
    except:
        return
# mania_contents('https://mania.kr/g2/bbs/board.php?bo_table=freetalk&wr_id=4898260')

In [83]:
def mlbpark_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    try:
        html_source = driver.page_source
        soup = bs(html_source, 'html.parser')
    except:
        return
    
    try:
        title = soup.select('.titles')[0].text.strip()
        review = soup.select('#contentDetail')[0].text.strip()

        iscomment = soup.select('.re_txt')
        if len(iscomment) != 0:
                box = []
                for i in iscomment:
                    box.append(i.text.replace('\n',''))
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
    except:
        return
# mlbpark_contents('https://mlbpark.donga.com/mp/b.php?p=841&b=bullpen&id=202101290051622547&select=&query=&user=&site=donga.com&reply=&source=&pos=&sig=h6j9Gf-YjhTRKfX2h4a9Rg-Akhlq')

In [84]:
def dcinside_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    try:
        html_source = driver.page_source
        soup = bs(html_source, 'html.parser')
    except:
        return
    
    try:
        title = soup.select('.title_subject')[0].text.strip()
        review = soup.select('.write_div')[0].text.strip()

        iscomment = soup.select('.usertxt.ub-word')
        if len(iscomment) != 0:
                box = []
                for i in iscomment:
                    box.append(i.text)
        else:
            box = []
            pass

        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)
    except:
        return
# dcinside_contents('https://gall.dcinside.com/board/view/?id=smartphone&no=6839316')

In [85]:
def ppomppu_contents(url, time, age, commu):
    driver.get(url)
    driver.implicitly_wait(30)
    
    try:
        html_source = driver.page_source
        soup = bs(html_source, 'html.parser')
    except:
        return
    
    try:
        title = soup.select('.view_title2')[0].text.strip()
        review = soup.select('.board-contents')[0].text.strip()
        iscomment = soup.select('.ori_comment')
        if len(iscomment) != 0:
                box = []
                for i in iscomment:
                    box.append(i.text.replace('<p>','').replace('\xa0','').replace('</p>','').replace('\n',''))
        else:
            box = []
            pass
        titles.append(title)
        reviews.append(review)
        comments.append(box)
        times.append(time)
        ages.append(age)
        commus.append(commu)

    except:
        return
# ppomppu_contents('https://www.ppomppu.co.kr/zboard/view.php?id=freeboard&divpage=1&no=7322993')

In [86]:
import time
def daum_cafe_contents(url, timee, age, commu):
    driver.get(url) #해당 url로 이동
    driver.implicitly_wait(30)
    
    try:
        driver.switch_to.frame('down') 
        time.sleep(2)
        soup = bs(driver.page_source, 'lxml') 
        
        try:
            title = soup.find('strong',class_='tit_info').text.strip() #제목 찾아서 수집
            review = soup.select("#user_contents")[0].text.strip()
            iscomment = soup.select(".original_comment")

            if len(iscomment) != 0:
                box = []
                for i in iscomment:
                    box.append(i.text)
            else:
                box = []
                pass

            titles.append(title)
            reviews.append(review)
            comments.append(box)
            times.append(timee)
            ages.append(age)
            commus.append(commu)

        except:
            return

        
    except UnexpectedAlertPresentException as e:
        return
    
    except NoSuchFrameException as e:
        return
    
    except NoAlertPresentException as e:
        return
    
    except:
        time.sleep(2)
        alert = driver.switch_to.alert
        alert.dismiss()
        return
    
# daum_cafe_contents('http://cafe.daum.net/_c21_/bbs_read?grpid=7SK&fldid=2tP1&datanum=204437', '1','1')
# daum_cafe_contents('https://cafe.daum.net/WorldcupLove/IC6M/11858677','1','1')


In [87]:
def pann_nate_contents(url, time, age, commu):
    driver.get(url) #해당 url로 이동
    driver.implicitly_wait(30)
    
    try:
        html_source = driver.page_source
        soup = bs(html_source, 'html.parser')
        try:
            title = soup.select(".post-tit-info > h4")[0].text.strip()
            review = soup.select("#contentArea")[0].text.strip()
            iscomment = soup.select(".usertxt")

            if len(iscomment) != 0:
                box = []
                for i in iscomment:
                    box.append(i.text.replace("\t","").replace("\n",""))
            else:
                box = []
                pass

            titles.append(title)
            reviews.append(review)
            comments.append(box)
            times.append(time)
            ages.append(age)
            commus.append(commu)
        except:
            return
        
    except UnexpectedAlertPresentException as e:
        return
    
    except:
        time.sleep(2)
        alert = driver.switch_to.alert
        alert.dismiss()
        return
    
# pann_nate_contents('https://pann.nate.com/talk/357631397', '1','1')


In [88]:
def instiz_contents(url, time, age, commu):
    driver.get(url) #해당 url로 이동
    driver.implicitly_wait(30)

    try:
        html_source = driver.page_source
        soup = bs(html_source, 'html.parser')
        
        try:
            title = soup.select("#nowsubject > a")[0].text.strip()
            review = soup.select("#memo_content_1")[0].text.strip().replace('\xa0','').replace('\n','')
            iscomment = soup.select(".comment_line")

            if len(iscomment) != 0:
                box = []
                for i in iscomment:
                    minitext_element = i.select(".minitext")[0]
                    minitext_element.extract()
                    box.append(i.text.split("\n")[0])
            else:
                box = []
                pass

            titles.append(title)
            reviews.append(review)
            comments.append(box)
            times.append(time)
            ages.append(age)
            commus.append(commu)
        except:
            return
        
    except UnexpectedAlertPresentException as e:
        return
    
    except NoSuchFrameException as e:
        return
    
    except NoAlertPresentException as e:
        return
    
    except:
        time.sleep(2)
        alert = driver.switch_to.alert
        alert.dismiss()
        #         print("no alert")
        #         WebDriverWait(driver, 3).until(EC.alert_is_present())
        #         alert.accept()
        return

    
    
# instiz_contents('http://www.instiz.net/name/41368932?category=1', '1', '1')

In [89]:
def naver_cafe_contents(url, timee, age, commu):
    #수집한 url 이 담겨있는 a에서 for문 반복
    
    if url in ['https://cafe.naver.com/ArticleRead.nhn?clubid=27131930&userDisplay=50&boardtype=L&articleid=10491275&referrerAllArticles=false']:
        return
    
    driver.get(url) #해당 url로 이동
    driver.implicitly_wait(30)

    #프레임 전환
    try: 
        driver.switch_to.frame('cafe_main') 
        time.sleep(1)
        try:
            soup = bs(driver.page_source, 'html')
        
        except UnexpectedAlertPresentException as e:
            time.sleep(2)
            alert = driver.switch_to.alert
            alert.dismiss()
            return
    
        except NoSuchFrameException as e:
            return
    
        except NoAlertPresentException as e:
            return
        
             # 제목 수집
        try:
            title = soup.find('h3',class_='title_text').text.replace('\n','').replace('\r','').strip() #제목 찾아서 수집
            titles.append(title)
            times.append(timee)
            ages.append(age)
            commus.append(commu)
        except:
            tabs = driver.window_handles
            driver.switch_to.window(tabs[1])
            driver.close()
            driver.switch_to.window(tabs[0])
            return

            try:
                title = driver.find_element_by_css_selector('h3.title_text').text.strip() #위 코드가 작동 안하는 경우가 있어서 같은 동작 다른 코드
                titles.append(title)
                time.sleep(1)
            except:
                title = [] #위 두개 코드로도 작동이 안되면 빈 리스트로 두고 다음 게시물로 넘어가도록 함
                titles.append(title)  

            # 본문 수집
        try:
            content = soup.select_one('div.se-module.se-module-text').text.strip() #본문 찾아서 수집
            reviews.append(content)
        except :
            try:
                content = soup.select_one('div.ContentRenderer').text.strip() #위 코드 작동 안하는 경우가 있어서 다른 코드
                reviews.append(content)
            except :
                try: 
                    content = driver.find_element_by_css_selector('div.se-module.se-module-text').text.strip() #위 코드 작동 안하는 경우 대비 다른 코드
                    reviews.append(content)
                except :
                    try:
                        content = driver.find_element_by_css_selector('div.ContentRenderer').text.strip() #위 코드 작동 안하는 경우 대비 다른 코드
                        reviews.append(content)
                    except :
                        content = [] #빈 리스트로 두고 다음 게시물로 넘어가도록 함
                        reviews.append(content)

        # 댓글 수집
        iscomment = soup.find_all('span',class_='text_comment')
        if len(iscomment) != 0: # 댓글이 0개가 아니면 
            box = []
            for i in iscomment:
                box.append([i.get_text()])
        else: # 댓글이 0개면 넘어감
            box = []
            pass
        comments.append(box)

    except UnexpectedAlertPresentException as e:
        time.sleep(2)
        alert = driver.switch_to.alert
        alert.dismiss()
        return
    
    except NoSuchFrameException as e:
        return
    
    except NoAlertPresentException as e:
        return
    
    except:
        time.sleep(2)
        alert = driver.switch_to.alert
        alert.dismiss()
        return
#         print("no alert")
#         WebDriverWait(driver, 3).until(EC.alert_is_present())
#         alert.accept()
        
            
# naver_cafe_contents('https://cafe.naver.com/ArticleRead.nhn?clubid=10298136&userDisplay=50&boardtype=L&articleid=27368505&referrerAllArticles=false', '1', '1', '1')
# naver_cafe_contents('https://cafe.naver.com/ArticleRead.nhn?clubid=29465158&userDisplay=50&boardtype=L&articleid=4737083&referrerAllArticles=false', '1', '1')
# naver_cafe_contents('https://cafe.naver.com/lolkor?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D19543191%2526userDisplay%3D50%2526boardtype%3DL%2526articleid%3D26770652%2526referrerAllArticles%3Dfalse', '1', '1')

------------------------------
#### <b>Main문</b>

In [132]:
chrome_driver = './chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
driver = webdriver.Chrome(chrome_driver, chrome_options=options)
driver.implicitly_wait(1) # sleep()과 차이점은 sleep()은 지정된 시간 동안 무조건 기다리라는 뜻이고, 암시적 대기는 지정한 시간이 되기 전에 페이지 로드가 끝나면 다음 작업을 진행하라

In [170]:
len(collected_community_df)

17302

In [137]:
sample = rest_url_df[2000:]
len(sample)

1116

In [91]:
sample = collected_community_df[200:300]
len(sample)

100

In [99]:
sample[492:497]

,No.,채널,날짜,작성자,제목,URL,세대
1083,229,오늘의 집 커뮤니티,20210401,NaN,이미지로 찾기 \r\n비슷한 \r\n비슷한 \r\n비슷한 \r\n비슷한 \r\n오랜...,https://ohou.se/contents/card_collections/7454...,2030대
1084,230,오늘의 집 커뮤니티,20210401,NaN,이미지로 찾기 \r\n비슷한 \r\n비슷한 \r\n비슷한 \r\n비슷한 \r\n거실...,https://ohou.se/contents/card_collections/7446...,2030대
1086,232,인벤,20210401,오징****,https://youtu.be/6BDaf3DwzRU\r\n애기들 그릇에 뽀로로까지 ...,http://www.inven.co.kr/board/webzine/2097/1610...,2030대
1089,3,slr클럽,20210531,느리*****,최근에야 로봇청소기를 들여 사용하고 있습니다. \r\n기존에 한 3년도 더전에 쓰다...,http://www.slrclub.com/bbs/vx2.php?id=free&no=...,2030대
1090,4,오늘의 집 커뮤니티,20210531,NaN,.css-s07o4k-CardDescription{display:block;marg...,https://ohou.se/contents/card_collections/8276...,2030대


In [98]:
collected_community_df[490:497]

,No.,채널,날짜,작성자,제목,URL,세대
491,47,다이렉트 결혼준비,20210222,혜롱******,\r\n \r\n \r\n \r\n \r\n \r\n안녕하세요~~ ^^ \r...,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2030대
492,48,다이렉트 결혼준비,20210222,미라*,\r\n \r\n \r\n \r\n \r\n \r\n손목이 자주 아픈 편이어...,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2030대
493,49,다이렉트 결혼준비,20210222,*,\r\n \r\n \r\n \r\n \r\n \r\n식기세척기 주말에만 사용...,https://cafe.naver.com/ArticleRead.nhn?clubid=...,2030대
494,50,더쿠,20210222,무명****,한 10년20년전에 쓸땐 기름도 다 잘 안닦여서 2차세척 필수였는데\r\n\r\n요...,https://theqoo.net/index.php?mid=review&filter...,2030대
495,51,네이트 판2,20210222,NaN,제가 한 이것저것 다 만드는 카페에서 일을 하고 있습니다. \r\n현재 2일간 일을...,https://pann.nate.com/talk/357903843,2030대
496,52,오늘의 집 커뮤니티,20210222,NaN,식기세척기 구매하려고 하는데 색상이랑 위치가 고민이 되요...\r\n1. 정수기밑\...,https://ohou.se/questions/71269?affect_id=0&af...,2030대
497,53,오늘의 집 커뮤니티,20210222,NaN,이미지로 찾기 \r\n비슷한 \r\n주방구조가 ㄷ자 공간이 나오게하려면 인테리어 공...,https://ohou.se/contents/card_collections/6853...,2030대


In [138]:
from tqdm import tqdm
import time
#제목, 본문, 댓글, 날짜 빈 list 생성
titles = []
reviews = []
comments = []
times = []
ages = []
commus = []

# 커뮤니티 URL마다 loop
for i in tqdm(sample.index):         #변수 수정
    timee = sample.loc[i,'날짜']       # 변수 수정
    age= sample.loc[i,'세대']        # 변수 수정
    community_url = sample.loc[i,'URL']         # 변수 수정
    community = community_url[:community_url.find('/',8)]
#     print(community_url)
    
    if community in ['http://cafe.naver.com', 'https://cafe.naver.com']:
        naver_cafe_contents(community_url, timee, age, '네이버카페')
    elif community in ['http://www.instiz.net','https://www.instiz.net']:
        instiz_contents(community_url, timee, age, '인스티즈')
    elif community in ['http://pann.nate.com', 'https://pann.nate.com'] :
        pann_nate_contents(community_url, timee, age, '네이트판')
    elif community in ['http://cafe.daum.net', 'https://cafe.daum.net']:
        daum_cafe_contents(community_url, timee, age, '다음카페')
    elif community in ['http://www.ppomppu.co.kr', 'https://www.ppomppu.co.kr']:
        ppomppu_contents(community_url, timee, age, '뽐뿌')
    elif community in ['http://gall.dcinside.com', 'https://gall.dcinside.com']:
        dcinside_contents(community_url, timee, age, '디씨인갤러리')
    elif community in ['http://mlbpark.donga.com', 'https://mlbpark.donga.com']:
        mlbpark_contents(community_url, timee, age, 'MLB파크')
    elif community in ['http://nbamania.com', 'https://nbamania.com']:
        mania_contents(community_url, timee, age, 'NBA마니아')
    elif community in ['http://web.humoruniv.com', 'https://web.humoruniv.com']:
        humoruniv_contents(community_url, timee, age, '유머대학')
    elif community in ['http://www.slrclub.com', 'https://www.slrclub.com']:
        slr_contents(community_url, timee, age, 'SLR클럽')
    elif community in ['http://www.bobaedream.co.kr', 'https://www.bobaedream.co.kr']:
        bobae_contents(community_url, timee, age, '보배드림')
    elif community in ['http://theqoo.net', 'https://theqoo.net']:
        theqoo_contents(community_url, timee, age, '더쿠')
    elif community in ['http://www.gasengi.com', 'https://www.gasengi.com']:
        gasengi_contents(community_url, timee, age, '가생이')
    elif community in ['http://www.inven.co.kr', 'https://www.inven.co.kr']:
        inven_contents(community_url, timee, age, '인벤')
    elif community in ['http://hiphople.com', 'https://hiphople.com']:
        hiphople_contents(community_url, timee, age, '힙합엘이')
    elif community in ['http://forum.blackdesertm.com', 'https://forum.blackdesertm.com']:
        blackdesertm_contents(community_url, timee, age, '검은사막')
    elif community in ['http://ohou.se', 'https://ohou.se']:
        ohou_contents(community_url, timee, age, '오늘의집')
    elif community in ['http://www.dmitory.com', 'https://www.dmitory.com']:
        dmitory_contents(community_url, timee, age, '디미토리')
    elif community in ['http://bbs.ruliweb.com', 'https://bbs.ruliweb.com']:
        ruliweb_contents(community_url, timee, age, '루리웹')
    elif community in ['http://www.etoland.co.kr', 'https://www.etoland.co.kr']:
        etoland_contents(community_url, timee, age, '이토랜드')
    elif community in ['http://m.clien.net', 'https://m.clien.net']:
        clien_contents(community_url, timee, age, '클리엔')
    elif community in ['http://www.mimint.co.kr', 'https://www.mimint.co.kr']:
        mimint_contents(community_url, timee, age, '미민트')
    elif community in ['http://www.todaysppc.com', 'https://www.todaysppc.com']:
        todaysppc_contents(community_url, timee, age, '투데이스피피씨')
    elif community in ['http://www.todayhumor.co.kr', 'https://www.todayhumor.co.kr']:
        todayhumor_contents(community_url, timee, age, '오늘의유머')
    elif community in ['http://eunpyeong.momscafe.net', 'https://eunpyeong.momscafe.net']:
        eunpyeong_contents(community_url, timee, age, '은평구맘카페')
    elif community in ['http://www.82cook.com', 'https://www.82cook.com']:
        cook82_contents(community_url, timee, age, '쿡82')
    elif community in ['http://www.ddanzi.com', 'https://www.ddanzi.com']:
        ddanzi_contents(community_url, timee, age, '딴지')
    elif community in ['http://dvdprime.com', 'https://dvdprime.com']:
        dprime_contents(community_url, timee, age, 'dvd프라임')
    elif community in ['http://www.koreapork.or.kr', 'https://www.koreapork.or.kr']:
        koreapork_contents(community_url, timee, age, '한돈')
    elif community in ['http://www.goso.co.kr', 'https://www.goso.co.kr']:
        goso_contents(community_url, timee, age, '소비자고발')
    elif community in ['http://stock.thinkpool.com', 'https://stock.thinkpool.com']:
        thinkpool_contents(community_url, timee, age, '띵크풀')
    elif community in ['http://www.dealbada.com', 'https://www.dealbada.com']:
        dealbada_contents(community_url, timee, age, '딜바다')
    elif community in ['http://soccerline.kr', 'https://soccerline.kr']:
        soccerline_contents(community_url, timee, age, '싸커라인')
    elif community in ['https://www.fmkorea.com', 'https://www.fmkorea.com']:
        fmkorea_contents(community_url, timee, age, 'FM코리아')
    else:
        print('No Community')

 51%|██████████████████████████████████████                                     | 566/1116 [1:00:29<1:35:29, 10.42s/it]

No Community


 97%|█████████████████████████████████████████████████████████████████████████▊  | 1083/1116 [1:52:40<02:26,  4.45s/it]

No Community


 98%|██████████████████████████████████████████████████████████████████████████▏ | 1089/1116 [1:52:47<00:47,  1.75s/it]

No Community


100%|████████████████████████████████████████████████████████████████████████████| 1116/1116 [1:55:19<00:00,  6.20s/it]


In [139]:
data = {
    "TITLES": titles,
    "REVIEW": reviews,
    "COMMENTS": comments,
    "TIME": times,
    "AGE": ages,
    "COMMUNITY": commus
}
community_review_df = pd.DataFrame(data)

In [140]:
community_review_df

,TITLES,REVIEW,COMMENTS,TIME,AGE,COMMUNITY
0,식기세척기...,"사려고하는 식기세척기 빌트인 사이즈가 820mm(세로기준) 인데,\n저희집 씽크대는...",[대리석 상판이 인조대리석이면싱크대 걸레받이 빼서싱크대 장 다리 전체적으로 올리면잘...,20210519,3040대,뽐뿌
1,비대면 견적 가능한 곳 찾고 있습니다. 오픈점 등 소개 부탁드려요!,"안녕하세요. \n \n한동안 가전 살일이 없어서 지나가면서 ""이렇게들 구매하시는구나...",[정보쪽지로 보내드립니다 부디잘견적나오시길...구성은 할인율좋은구성은아니네욥...ㅜ...,20210519,3040대,뽐뿌
2,집안에 식기세척기 이론 방법이 아딸지,1. 현재 식기세척기\n물을 뿌려 세척\n이용해 다시 재 보관\n \n아이디어\n1...,[],20210519,3040대,보배드림
3,식기세척기에 코펠 넣으면 망하는군요 ㅠㅠㅋ,주말에 오랜만에 캠핑예정이라\n준비겸해서 코펠좀 씻어두려고\n식기세척기에 돌렸더니\...,"[코팅제품은 잘못넣으면 코팅을 세척해버리죠 ㅋㅋ, 네.. 그렇게 되어버렸네요.. ㅠ...",20210518,3040대,클리엔
4,식기세척기를 싸게 구입했으나 추가 지출이 생기는군요 ㅠㅠ,식기세척기를 싸게 구입했으나 추가 지출이 생기는군요 \n\n60cm 확보가 되어야 ...,"[엘지 식세기 사셨나요?, 네 지난번 스마일세일? 지마켓에서 샀어요^^, 식세기 사...",20210518,3040대,클리엔
...,...,...,...,...,...,...
711,로봇 청소기 새로 바꿨습니다. 신세계네요.!!(내돈내산후기),원래 쓰던 모델이 샤오미 1세대인데 2017년초쯤 샀던 걸로 기억합니다.\n그후로 ...,[LG 물걸레만 되는걸로 살까말까 고민중인데겸용으로 해야 하나 로보락으로 해야하나 ...,20211009,연령불가,dvd프라임
712,초음파 식기세척기 업체의 사기성 계약,이노클린이라는 초음파식기세척기 업체에서\n금융할부라고만 하고 11월3일에 계약을하고...,[],20211117,연령불가,소비자고발
713,이게 있으면 인생 효율이 올라간다 하는 전자기기/제품 있는지요?,"안녕하세요연말 선물용으로 구입하려는데요(최대 40만원 이내)이게 있으면 인생 효율,...","[구글홈과 전자기기 연결해서 음성으로 제어하면 참 편하더군요.집에서 전등,청소기,에...",20211110,연령불가,dvd프라임
714,로봇 청소기 만족도 말인데요?,37평 이사 선물로 받은 아이로봇이라는 로봇청소기 제품을 처음 써보게 되었습니다.\...,"[S7급 이상으로 사야죠., 먼지 흡입하는 용도의 로봇 청소기는 제 지인들 추천템은...",20211101,연령불가,dvd프라임


In [141]:
community_review_df.to_csv('./식세기 원문/rest_2000_3336.csv', encoding='utf-8 sig', index=False)

------------------------

In [160]:
file1 = pd.read_csv('./식세기 원문/rest_1_300.csv', encoding='utf-8 sig')
file2 = pd.read_csv('./식세기 원문/rest_300_600.csv', encoding='utf-8 sig')
file3 = pd.read_csv('./식세기 원문/rest_600_1000.csv', encoding='utf-8 sig')
file4 = pd.read_csv('./식세기 원문/rest_1000_1500.csv', encoding='utf-8 sig')
file5 = pd.read_csv('./식세기 원문/rest_1500_2000.csv', encoding='utf-8 sig')
file6 = pd.read_csv('./식세기 원문/rest_2000_3336.csv', encoding='utf-8 sig')

fin_file = pd.concat([file1, file2, file3, file4, file5, file6])
fin_file = fin_file.dropna()
fin_file.to_csv('./식세기 원문/dishwasher_rest_community_origin.csv', encoding='utf-8 sig', index=False)

In [161]:
len(fin_file)

1985

In [173]:
search_words = ['퐁퐁남','퐁퐁이','퐁퐁부인','퐁퐁단','설거지단','내무부장관','설거지론','진중권','문재인','윤석열','알바','어쩔티비','뉴이스트','정인이','한녀','한남',\
                '맥도날드','스시녀','창녀','창년','짤녀']
seolgeoji_title_file = fin_file[fin_file['TITLES'].str.contains('|'.join(search_words))]
seolgeoji_review_file = fin_file[fin_file['REVIEW'].str.contains('|'.join(search_words))]
# seolgeoji_file.to_csv('./식세기 원문/except_seolgeoji_file.csv', encoding='utf-8 sig', index=False)
except_index = pd.concat([seolgeoji_title_file, seolgeoji_review_file]).drop_duplicates(['TITLES','REVIEW'], keep='first').index.to_list()
except_index.sort()

except_seolgeoji_df = fin_file.drop(except_index, axis=0)

In [174]:
except_seolgeoji_df.to_csv('./식세기 원문/except_seolgeoji_file.csv', encoding='utf-8 sig', index=False)